In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
inspector = inspect(engine)

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine, reflect=True)

In [12]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [19]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [21]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [36]:
#Sort the Dataframe values by date
sorted_df = last_year_df.sort_values(by = "date")
sorted_df.head()

NameError: name 'last_year_df' is not defined

# Exploratory Precipitation Analysis

In [38]:
# Find the most recent date in the data set.
engine.execute('SELECT max(date) FROM Measurement').fetchall()

[('2017-08-23',)]

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
# Sort the dataframe by date
# Save the query results as a Pandas DataFrame. Explicitly set the column names
# Use Pandas Plotting with Matplotlib to plot the data

In [39]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
inspector.get_table_names()
columns = inspector.get_columns('measurement')
#Select only the "date" and "prcp" values
session.query(Measurement.date, Measurement.prcp).all()

[('2010-01-01', 0.08),
 ('2010-01-02', 0.0),
 ('2010-01-03', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-06', None),
 ('2010-01-07', 0.06),
 ('2010-01-08', 0.0),
 ('2010-01-09', 0.0),
 ('2010-01-10', 0.0),
 ('2010-01-11', 0.01),
 ('2010-01-12', 0.0),
 ('2010-01-14', 0.0),
 ('2010-01-15', 0.0),
 ('2010-01-16', 0.0),
 ('2010-01-17', 0.0),
 ('2010-01-18', 0.0),
 ('2010-01-19', 0.0),
 ('2010-01-20', 0.0),
 ('2010-01-21', 0.0),
 ('2010-01-22', 0.0),
 ('2010-01-23', 0.0),
 ('2010-01-24', 0.01),
 ('2010-01-25', 0.0),
 ('2010-01-26', 0.04),
 ('2010-01-27', 0.12),
 ('2010-01-28', 0.0),
 ('2010-01-30', None),
 ('2010-01-31', 0.03),
 ('2010-02-01', 0.01),
 ('2010-02-03', None),
 ('2010-02-04', 0.01),
 ('2010-02-05', 0.0),
 ('2010-02-06', 0.0),
 ('2010-02-07', 0.0),
 ('2010-02-08', 0.0),
 ('2010-02-09', 0.0),
 ('2010-02-11', 0.0),
 ('2010-02-12', 0.02),
 ('2010-02-13', 0.01),
 ('2010-02-14', 0.0),
 ('2010-02-15', 0.0),
 ('2010-02-16', 0.0),
 ('2010-02-17', 0.0),
 ('2010-02-19', None),
 ('2010-02-20', 0

In [65]:
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first
recent_date

recent_date_dt = dt.datetime.strptime(recent_date, "%Y-%m-%d")

one_yr_earlier = recent_date_dt - dt.timedelta(days=365)
one_yr_earlier_dt = dt.datetime.strptime(one_yr_earlier, "%Y-%m-%d")

prcp_scores_query = session.query(Measurement.date,Measurement.prcp).\
    filter(Measurement.date > one_yr_earlier_dt).\
    order_by(Measurement.date).all()
# Save the query results as a Pandas DataFrame. Explicitly set the column names
data = pd.DataFrame(prcp_scores_query, columns=['Date', 'Precipitaion'])
data.set_index('Date', inplace=True)

TypeError: strptime() argument 1 must be str, not method

In [47]:
#Sort the Dataframe values by date
data = pd.DataFrame.sort_index(data)

NameError: name 'data' is not defined

In [37]:
# Use Pandas to calculate the summary statistics for the precipitation data
plt.figure(figsize=(15, 10))
plt.plot(sorted_df.index, sorted_df['prcp'])

plt.legend(['precipitation'], loc="upper right", fontsize=15)
xloc = ['2016-08-23', '2016-10-01', '2016-11-09', '2016-12-19', '2017-01-28', '2017-03-09', '2017-04-18', '2017-05-31', '2017-07-10']

plt.xticks(xloc, rotation='vertical', fontsize=15)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Inches', fontsize=15)
plt.title(f"Precipitation ( inches) in Honolulu, Hawaii from \n {query_date} to {last}")
plt.yticks(size=15)

plt.xlim(0,370)
plt.ylim(-0.4,7)

plt.savefig("Images/Precipitation_Plot.png")

plt.show()
plt.tight_layout()




NameError: name 'sorted_df' is not defined

<Figure size 1500x1000 with 0 Axes>

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
station_count = session.query(func.count(Station.station)).first()[0]
station_count

NameError: name 'Station' is not defined

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_count = session.query(Measurement.station, func.count(Measurement.station))


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close Session

In [ ]:
# Close Session
session.close()